In [102]:
import os
import pandas as pd

# Read the data
df = pd.read_csv("data" + os.sep + "mbti_1.csv")
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [103]:
#extroverted phrases & engrams

#TODO Add Features before preprocessing

#CLASS WEIGHTS FOR RANDOM FORESTS verwenden statt oversampling / balancing 
# Ein neuronales Netz mit vier Ausgabe Neuronen 
#Topic Model Trainier
#F1 Score Introvertiert kann ich besonders schlecht vorhersagen macht das Sinn überhaupt?
#Topic Model, Sentence Encoder, Feature Gewicht bei XGBoost, Random Forest. 
#Confusion Matrix genaueres Bild, aber natrlich nicht nur auf einen Wert festmachen würde mehrere verwenden und 
#Hyperparameter Tuning mit GridSearchCV Zum Beispiel. 
#Toppic MOdel Trainieren 


#Präsentation eher auf Features Eingehen was mehr Korelleiert und vielleicht auf beispiele bringen. 



##Surface Realisation Features
#Feature Extroverts: "hi" at the beginning of a post
#Feature Introverts: "hello" at the beginning of a post
#Feature Extroverts: Multiple Exclamation Marks "*![opt. whitespace]!*" (STRONGLY CORELLATED WITH EXTROVERTED)
#Feature Introverts: Multiple full stops "*.[opt. whitespace].*"

#With Rel.F Ratio > 1.0

#Features bigrams for Extroverts
extroverted_bigrams = [

    "looking forward",
    "it was",
    "next week",
    "a bit",
    "up with",
    "will be",
    "i was",
    "at the",
    "to see",
    "which is",
    "for a",
    "i have",
    "ill be",
    "a good",
    "we went",
    "want to",
    "need to",
    "able to"
]

#Features stylistic_impressions
extroverted_stylistic_impressions = [
    "catch up",
    "take care"
]

#Feature Quantifiers (add them to one)
#Introverts generally tend to show a preference for a greater use of quantifiers upper four also uniquely introverted
interoverted_quantifiers = [
    "all the",
    "one of",
    "lots of",
    "loads of",  
    "a lot",
    "a few",
]



#Feature First_Person_singular Pronom
#Introverts make extensive use of first person singular pronoun
introverted_first_person_singular_pronoun = [
    "i can",
    "ive got",
    "ive done",
    "i think",
    "i am",
    "i",
]

#Feature: NegationCount
#Negations are more common in introverts
introverted_negations = [
    "dont",
    "cant"
    "wont"
]

#Feature: Word Count per Post. Extroverts tend to write longer posts

#Message Introverts show perefernce for coordinating conjunctions "and" and "but" whilst
#Extraverts uniquely show use of the ksubordinating "which"

#Feature ordinating conjuctions (and, but)
#Feature subordinating conjuctions(which)

In [104]:
#Multiple Exclamation Marks
#regex match more than 2 exclamation marks

import re
import tldextract


def convert_posts_to_list(post: str) -> list:
    return post.split("|||")


def avg_word_count(posts: list) -> float:
    return sum([len(post.split()) for post in posts]) / len(posts)


#Returns the avg count of 2 or more exclamation marks behind each other
def avg_exclamation_mark_count(posts: list) -> float:
    return sum([len(re.findall(r"!{2,}", post)) for post in posts]) / len(posts)


#Returns the avg count of 2 or more full stops behind each other
def avg_full_stop_count(posts: list) -> float:
    return sum([len(re.findall(r"\.{2,}", post)) for post in posts]) / len(posts)


def replace_url_with_domain(posts: list) -> str:
    
    for i, post in enumerate(posts):
        urls = re.findall(r"(https?://[^\s]+)", post)
        for url in urls:
            domain = tldextract.extract(url).domain
            post = post.replace(url, domain)
        posts[i] = post

    return posts


#remove trailing 3 dots from posts
def remove_trailing_3_dots(posts: list) -> str:
    
    # check if last 3 chars are "..."
    for i, post in enumerate(posts):
        #Posts in the dataset are cut off after 192 characters but not exactly, only after the next word is finished. The cut off is indicated by 3 dots at the end of the post.
        if len(post) >= 192 and post[-3:] == "...":
            posts[i] = post[:-3]

    return posts


def avg_count_of_hello(posts: list) -> float:
    return sum([len(re.findall(r"hello ", post)) for post in posts]) / len(posts)

def avg_count_of_hi(posts: list) -> float:
    return sum([len(re.findall(r"hi ", post)) for post in posts]) / len(posts)
    
def correct_expressive_lengthening(posts: list) -> list:
    for i, post in enumerate(posts):
        #correct expressive lengthening

        post = re.sub(r"([a-zA-Z])\1{2,}", r"\1", post)
        posts[i] = post

    return posts

#remove non alphabetical characters
def remove_non_alphabetical_characters(posts: list) -> list:
    for i, post in enumerate(posts):
        post = re.sub(r"[^a-zA-Z ]", "", post)
        posts[i] = post

    return posts



#average count of extroverted bigrams in posts
def avg_count_of_extroverted_bigrams(posts: list) -> float:
    return sum([sum([post.count(bigram) for bigram in extroverted_bigrams]) for post in posts]) / len(posts)

def avg_count_of_extroverted_stylistic_impressions(posts: list) -> float:
    return sum([sum([post.count(stylistic_impression) for stylistic_impression in extroverted_stylistic_impressions]) for post in posts]) / len(posts)

#average count of interoverted quantifiers
def avg_count_of_interoverted_quantifiers(posts: list) -> float:
    return sum([sum([post.count(quantifier) for quantifier in interoverted_quantifiers]) for post in posts]) / len(posts)

#average count of introverted first person singular pronoun
def avg_count_of_introverted_first_person_singular_pronoun(posts: list) -> float:
    return sum([sum([post.count(pronoun) for pronoun in introverted_first_person_singular_pronoun]) for post in posts]) / len(posts)

#average count of introverted negations
def avg_count_of_introverted_negations(posts: list) -> float:
    return sum([sum([post.count(negation) for negation in introverted_negations]) for post in posts]) / len(posts)


In [105]:
df["posts"] = df["posts"].apply(convert_posts_to_list)
df["avg_word_count"] = df["posts"].apply(avg_word_count)
df.head()

,type,posts,avg_word_count
0,INFJ,"['http://www.youtube.com/watch?v=qsXHcwe3krw, ...",12.10
1,ENTP,['I'm finding the lack of me in these posts ve...,24.38
2,INTP,['Good one _____ https://www.youtube.com/wa...,17.70
3,INTJ,"['Dear INTP, I enjoyed our conversation the ...",22.26
4,ENTJ,"['You're fired., That's another silly misconce...",20.32


In [106]:
df["posts"] = df["posts"].apply(remove_trailing_3_dots)
df["posts"] = df["posts"].apply(replace_url_with_domain)
df["avg_exclam_count"] = df["posts"].apply(avg_exclamation_mark_count)
df["avg_fullstop_count"] = df["posts"].apply(avg_full_stop_count)
df.head()



,type,posts,avg_word_count,avg_exclam_count,avg_fullstop_count
0,INFJ,"['youtube, tumblr, enfp and intj moments yout...",12.10,0.0,0.16
1,ENTP,['I'm finding the lack of me in these posts ve...,24.38,0.0,0.04
2,INTP,"['Good one _____ youtube, Of course, to whi...",17.70,0.0,0.08
3,INTJ,"['Dear INTP, I enjoyed our conversation the ...",22.26,0.0,0.14
4,ENTJ,"['You're fired., That's another silly misconce...",20.32,0.0,0.08


In [107]:
#Correct expressive lenghening
df["posts"] = df["posts"].apply(correct_expressive_lengthening)


In [108]:
df["avg_count_of_hello"] = df["posts"].apply(avg_count_of_hello)
df["avg_count_of_hi"] = df["posts"].apply(avg_count_of_hi)
df.head()

,type,posts,avg_word_count,avg_exclam_count,avg_fullstop_count,avg_count_of_hello,avg_count_of_hi
0,INFJ,"['youtube, tumblr, enfp and intj moments yout...",12.10,0.0,0.16,0.0,0.0
1,ENTP,['I'm finding the lack of me in these posts ve...,24.38,0.0,0.04,0.0,0.0
2,INTP,"['Good one _____ youtube, Of course, to whi...",17.70,0.0,0.08,0.0,0.0
3,INTJ,"['Dear INTP, I enjoyed our conversation the ...",22.26,0.0,0.14,0.0,0.0
4,ENTJ,"['You're fired., That's another silly misconce...",20.32,0.0,0.08,0.0,0.0


In [109]:
df["posts"] = df["posts"].apply(remove_non_alphabetical_characters)

df["avg_count_of_extroverted_bigrams"] = df["posts"].apply(avg_count_of_extroverted_bigrams)
df["avg_count_of_extroverted_stylistic_impressions"] = df["posts"].apply(avg_count_of_extroverted_stylistic_impressions)
df["avg_count_of_interoverted_quantifiers"] = df["posts"].apply(avg_count_of_interoverted_quantifiers)
df["avg_count_of_introverted_first_person_singular_pronoun"] = df["posts"].apply(avg_count_of_introverted_first_person_singular_pronoun)
df["avg_count_of_introverted_negations"] = df["posts"].apply(avg_count_of_introverted_negations)

df.head()

,type,posts,avg_word_count,avg_exclam_count,avg_fullstop_count,avg_count_of_hello,avg_count_of_hi,avg_count_of_extroverted_bigrams,avg_count_of_extroverted_stylistic_impressions,avg_count_of_interoverted_quantifiers,avg_count_of_introverted_first_person_singular_pronoun,avg_count_of_introverted_negations
0,INFJ,"[youtube, tumblr, enfp and intj moments youtu...",12.10,0.0,0.16,0.0,0.0,0.14,0.0,0.06,3.54,0.02
1,ENTP,[Im finding the lack of me in these posts very...,24.38,0.0,0.04,0.0,0.0,0.28,0.0,0.14,5.58,0.10
2,INTP,"[Good one youtube, Of course to which I sa...",17.70,0.0,0.08,0.0,0.0,0.26,0.0,0.00,5.18,0.04
3,INTJ,[Dear INTP I enjoyed our conversation the ot...,22.26,0.0,0.14,0.0,0.0,0.20,0.0,0.10,6.12,0.10
4,ENTJ,"[Youre fired, Thats another silly misconceptio...",20.32,0.0,0.08,0.0,0.0,0.14,0.0,0.00,6.10,0.10


In [110]:
df.describe()

,avg_word_count,avg_exclam_count,avg_fullstop_count,avg_count_of_hello,avg_count_of_hi,avg_count_of_extroverted_bigrams,avg_count_of_extroverted_stylistic_impressions,avg_count_of_interoverted_quantifiers,avg_count_of_introverted_first_person_singular_pronoun,avg_count_of_introverted_negations
count,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000,8675.000000
mean,26.128059,0.015914,0.205590,0.000679,0.001412,0.219699,0.000823,0.086849,7.086543,0.117837
std,5.787802,0.043035,0.228299,0.005661,0.008141,0.103800,0.004371,0.057216,1.737905,0.074221
min,1.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.729565,0.000000,0.080000,0.000000,0.000000,0.140000,0.000000,0.040000,6.000000,0.060000
50%,26.980000,0.000000,0.140000,0.000000,0.000000,0.214286,0.000000,0.080000,7.250000,0.100000
75%,30.440000,0.020000,0.260000,0.000000,0.000000,0.280000,0.000000,0.120000,8.300000,0.160000
max,38.600000,0.840000,3.736842,0.320000,0.400000,2.000000,0.060000,0.440000,12.780000,0.600000


In [111]:
preprocessed_df = pd.read_csv("mbti_preprocessed_complete.csv", index_col=0)

In [112]:
preprocessed_df.head()

,type,posts,encodedType,preprocessed_posts,extro_intro,intu_obs,feel_think,prosp_judg
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,"['youtube', 'tumblr', 'enfp', 'intj', 'moment'...",0,1,1,0
1,ENTP,'I'm finding the lack of me in these posts ver...,3,"['im', 'finding', 'the', 'lack', 'of', 'post',...",1,1,0,1
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,"['good', 'one', 'youtube', 'of', 'course', 'i'...",0,1,0,1
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,"['dear', 'intp', 'i', 'enjoyed', 'conversation...",0,1,0,0
4,ENTJ,'You're fired.|||That's another silly misconce...,2,"['youre', 'fired', 'thats', 'another', 'silly'...",1,1,0,0


In [113]:
preprocessed_df.shape

(8675, 8)

In [115]:

#drop type and posts from df
df_to_merge = df.drop(["type", "posts"], axis=1)
df_to_merge.head()

,avg_word_count,avg_exclam_count,avg_fullstop_count,avg_count_of_hello,avg_count_of_hi,avg_count_of_extroverted_bigrams,avg_count_of_extroverted_stylistic_impressions,avg_count_of_interoverted_quantifiers,avg_count_of_introverted_first_person_singular_pronoun,avg_count_of_introverted_negations
0,12.10,0.0,0.16,0.0,0.0,0.14,0.0,0.06,3.54,0.02
1,24.38,0.0,0.04,0.0,0.0,0.28,0.0,0.14,5.58,0.10
2,17.70,0.0,0.08,0.0,0.0,0.26,0.0,0.00,5.18,0.04
3,22.26,0.0,0.14,0.0,0.0,0.20,0.0,0.10,6.12,0.10
4,20.32,0.0,0.08,0.0,0.0,0.14,0.0,0.00,6.10,0.10


In [116]:
#merge preprocessed_df with df_to_merge on index
preprocessed_feature_df = preprocessed_df.merge(df_to_merge, left_index=True, right_index=True)
preprocessed_feature_df.head()



,type,posts,encodedType,preprocessed_posts,extro_intro,intu_obs,feel_think,prosp_judg,avg_word_count,avg_exclam_count,avg_fullstop_count,avg_count_of_hello,avg_count_of_hi,avg_count_of_extroverted_bigrams,avg_count_of_extroverted_stylistic_impressions,avg_count_of_interoverted_quantifiers,avg_count_of_introverted_first_person_singular_pronoun,avg_count_of_introverted_negations
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,"['youtube', 'tumblr', 'enfp', 'intj', 'moment'...",0,1,1,0,12.10,0.0,0.16,0.0,0.0,0.14,0.0,0.06,3.54,0.02
1,ENTP,'I'm finding the lack of me in these posts ver...,3,"['im', 'finding', 'the', 'lack', 'of', 'post',...",1,1,0,1,24.38,0.0,0.04,0.0,0.0,0.28,0.0,0.14,5.58,0.10
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,"['good', 'one', 'youtube', 'of', 'course', 'i'...",0,1,0,1,17.70,0.0,0.08,0.0,0.0,0.26,0.0,0.00,5.18,0.04
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,"['dear', 'intp', 'i', 'enjoyed', 'conversation...",0,1,0,0,22.26,0.0,0.14,0.0,0.0,0.20,0.0,0.10,6.12,0.10
4,ENTJ,'You're fired.|||That's another silly misconce...,2,"['youre', 'fired', 'thats', 'another', 'silly'...",1,1,0,0,20.32,0.0,0.08,0.0,0.0,0.14,0.0,0.00,6.10,0.10


In [ ]:
preprocessed_feature_df.to_csv("mbit_preprocessed_features.csv")